## Zaawansowane Metody Inteligencji Obliczeniowej
# Zadanie domowe 2
### Prowadzący: Michał Kempka, Marek Wydmuch
### Autor: Daniel Zdancewicz 145317

## Wprowadzenie

Całe zadanie jest oparte o różne wersje środowiska `FrozenLake` ze znanej biblioteki OpenAI Gym (https://gym.openai.com), która agreguje różnego rodzaju środowiska pod postacią jednego zunifikowanego API.

Zapoznaj się z opisem środowiska (https://gym.openai.com/envs/FrozenLake-v0), a następnie zapoznaj się z kodem poniżej. Pokazuje on podstawy użytkowania API biblioteki Gym.

#### Uwaga: Możesz dowolnie modyfikować elementy tego notebooka (wstawiać komórki i zmieniać kod) o ile nie napisano gdzieś inaczej.

In [3]:
import numpy as np
# Zainstaluj bibliotekę OpenAI Gym w wersji 0.18.0
!pip install gym == 0.18.0

ERROR: Invalid requirement: '=='

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import gym

## Zad. 1 - Policy iteration + value iteration (10 pkt.)

W komórkach poniżej zaimplementuj algorytmy **iteracji polityki** oraz **iteracji wartości**, wyznaczające deterministyczną politykę dla środowiska FrozenLake.

Odpowiedź na pytania wykonując odpowiednie eksperymenty (zostaw output odpowiednich komórek na poparcie swoich twierdzeń):
- Jak zmiana współczynniku `gamma` wpływa na wynikową politykę?
- Jak stochastyczność wpływa na liczbę iteracji potrzebnych do zbiegnięcia obu algorytmów oraz wynikową politykę?

#### Uwaga: nie zmieniaj nazwy funkcji `policy_iteration` i `value_iteration`, ani ich argumentów. Nie dopisuj do komórek z funkcjami innego kodu. Może zdefiniować funkcje pomocnicze dla danej funkcji w tej samej komórce (sprawdzarka wyciągnie ze zgłoszonego notebooka wyłącznie komórki zawierającą funkcje `policy_iteration` i `value_iteration` do sprawdzenia, kod w innych komórkach nie będzie widziany przez sprawdzarkę!)

Odpowiedzi:
1. Współczynnik `gamma` ustawiony w przedziale 0,0-0,1 lub o wartości 1.0 powoduje błędne polityki o złych liczbach iteracji, które powodują przegraną ciągłą przegraną.
2. Stochastyczność wpływa na:
- Liczba iteracji potrzebnych do zbiegnięcia:
  - Stochastyczne — Potrzebujemy więcej iteracji przy większym gamma do zbiegnięcia. Może to wynikać ze względu na otrzebę uwzględnienia prawdopodobieństwa przejść pomiędzy stanami, co każe agentowi myśleć o ryzku związanymi z akcjami.
  - Deterministyczne — zbiegają szybciej ze względu na brak potrzeby przewidywania przez agenta ryzyka swoich akcji.
- Wynikowa polityka:
  - Stochastyczne — Zdaje się ostrożna, agent możliwie uwzględnia ryzyko związane ze ślizganiem się na lodzie. Przez co możliwie, żę będzie rozważać ścieżki, które prowadzą do celu z mniejszym ryzykiem, ale są dłuższe.
  - Deterministyczne — Agent może zawsze wybrać najkrótszą ścieżkę do celu ze względu na pewność wykonanych akcji przez agenta.

In [5]:
def evaluate_empirically(env, pi, episodes=1000, max_actions=100):
  mean_reward = 0
  for episode in range(episodes):
    state = env.reset()
    total = 0

    for _ in range(max_actions):
      state, reward, termination, _ = env.step(pi[state])
      total += reward
      if termination: break
    mean_reward = mean_reward + 1 / (episode + 1) * (total - mean_reward)
  return mean_reward

In [6]:
def policy_iteration(P, gamma, delta=0.001):
  """
  Argumenty:
      P — model przejścia, gdzie P[s][a] == [(probability, nextstate, reward, done), ...]
      gamma — współczynnik dyskontujący
      delta — tolerancja warunku stopu
  Zwracane wartości:
      V — lista o długości len(P) zawierający oszacowane wartość stanu s: V[s]
      pi — lista o długości len(P) zawierający wyznaczoną deterministyczną politykę - akcję dla stanu s: pi[s]
      i — ilość iteracji algorytmu po wszystkich stanach
  """
  def evaluate():
    while True:
      score = 0
      for state in states:
        value = V[state]
        action = policy[state]
        V[state] = sum(
          probability * (reward + gamma * V[next]) for probability, next, reward, _ in P[state][action]
        )
        score = max(score, abs(value - V[state]))
      if score < delta: break

  def improve():
    is_stable = True
    for state in states:
      previous = policy[state]
      actions = range(len(P[state]))
      policy[state] = max(actions, key=lambda action: sum(
        probability * (reward + gamma * V[next]) for probability, next, reward, _ in P[state][action]
      ))

      if previous != policy[state]: is_stable = False
    return is_stable

  V = [0] * len(P)
  policy = [0] * len(P)
  states = range(len(P))
  iterations = 0

  while True:
    iterations += 1
    evaluate()
    is_stable = improve()
    if is_stable: break

  return V, policy, iterations


In [7]:
env = gym.make('FrozenLake-v0', is_slippery=False)
P = env.P
gamma = 0.5
V, policy, iterations = policy_iteration(P, gamma)

print(f"""
Wartości stanów (V):
{V}
Polityka (pi):
{policy}
Liczba iteracji: {iterations}
Empiryczny wynik: {evaluate_empirically(env, policy)}
""")


Wartości stanów (V):
[0.03125, 0.0625, 0.125, 0.0625, 0.0625, 0.0, 0.25, 0.0, 0.125, 0.25, 0.5, 0.0, 0.0, 0.5, 1.0, 0.0]
Polityka (pi):
[1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0]
Liczba iteracji: 7
Empiryczny wynik: 1.0



In [8]:
def value_iteration(P, gamma, delta=0.001):
  """
  Argumenty:
      P — model przejścia, gdzie P[s][a] == [(probability, nextstate, reward, done), ...]
      gamma — współczynnik dyskontujący
      delta — tolerancja warunku stopu
  Zwracane wartości:
      Q — lista o długości len(P) zawierający listy z oszacowanymi wartościami dla stanu s i akcji a: Q[s][a]
      pi — lista o długości len(P) zawierający wyznaczoną deterministyczną politykę - akcję dla stanu s: pi[s]
      i — ilość iteracji algorytmu po wszystkich stanach
  """
  def evaluate():
    score = 0
    for state in states:
      for action in range(len(P[state])):
        q = Q[state][action]
        Q[state][action] = sum([p * (r + gamma * max(Q[s_])) for p, s_, r, _ in P[state][action]])
        score = max(score, abs(q - Q[state][action]))
    return score

  def extract():
    return [Q[state].index(max(Q[state])) for state in states]

  states = P.keys()
  Q = [[0] * len(P[state]) for state in states]
  iterations = 0

  while True:
    iterations += 1
    score = evaluate()
    if score < delta: break

  policy = extract()
  return Q, policy, iterations

In [9]:
env = gym.make('FrozenLake-v0', is_slippery=False)
P = env.P
gamma = 0.5
Q, policy, iterations = value_iteration(P, gamma)

print(f"""
Wartości stanów i akcji (Q):
{Q}
Polityka (pi):
{policy}
Liczba iteracji: {iterations}
Empiryczny wynik: {evaluate_empirically(env, policy)}
""")


Wartości stanów i akcji (Q):
[[0.015625, 0.03125, 0.03125, 0.015625], [0.015625, 0.0, 0.0625, 0.03125], [0.03125, 0.125, 0.03125, 0.0625], [0.0625, 0.0, 0.03125, 0.03125], [0.03125, 0.0625, 0.0, 0.015625], [0.0, 0.0, 0.0, 0.0], [0.0, 0.25, 0.0, 0.0625], [0.0, 0.0, 0.0, 0.0], [0.0625, 0.0, 0.125, 0.03125], [0.0625, 0.25, 0.25, 0.0], [0.125, 0.5, 0.0, 0.125], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.25, 0.5, 0.125], [0.25, 0.5, 1.0, 0.25], [0.0, 0.0, 0.0, 0.0]]
Polityka (pi):
[1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0]
Liczba iteracji: 8
Empiryczny wynik: 1.0



### Eksperyment wpływu `gamma`.

In [10]:
env = gym.make('FrozenLake-v0', is_slippery=False)
P = env.P
gammas = [0.0, 0.1, 0.5, 0.9, 0.99, 1]

print("Policy Iteration:")
for gamma in gammas:
  _, policy, iterations = policy_iteration(P, gamma)
  print(f"""
  Gamma: {gamma}
  Polityka (pi): {policy}
  Liczba iteracji: {iterations}
  Empiryczny wynik: {evaluate_empirically(env, policy)}
  """)

print("Value Iteration:")
for gamma in gammas:
  _, policy, iterations = value_iteration(P, gamma)
  print(f"""
  Gamma: {gamma}
  Polityka (pi): {policy}
  Liczba iteracji: {iterations}
  Empiryczny wynik: {evaluate_empirically(env, policy)}
  """)


Policy Iteration:

  Gamma: 0.0
  Polityka (pi): [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0]
  Liczba iteracji: 2
  Empiryczny wynik: 0.0
  

  Gamma: 0.1
  Polityka (pi): [1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0]
  Liczba iteracji: 7
  Empiryczny wynik: 1.0
  

  Gamma: 0.5
  Polityka (pi): [1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0]
  Liczba iteracji: 7
  Empiryczny wynik: 1.0
  

  Gamma: 0.9
  Polityka (pi): [1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0]
  Liczba iteracji: 7
  Empiryczny wynik: 1.0
  

  Gamma: 0.99
  Polityka (pi): [1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 0]
  Liczba iteracji: 7
  Empiryczny wynik: 1.0
  

  Gamma: 1
  Polityka (pi): [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]
  Liczba iteracji: 8
  Empiryczny wynik: 0.0
  
Value Iteration:

  Gamma: 0.0
  Polityka (pi): [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0]
  Liczba iteracji: 2
  Empiryczny wynik: 0.0
  

  Gamma: 0.1
  Polityka (pi): [0, 2, 1, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0,

### Eksperyment środowiska deterministycznego kontra stochastycznego.

In [11]:
env_deterministic = gym.make('FrozenLake-v0', is_slippery=False)
env_stochastic = gym.make('FrozenLake-v0', is_slippery=True)
P_deterministic = env_deterministic.P
P_stochastic = env_stochastic.P

gammas = [0.0, 0.1, 0.5, 0.9, 0.99, 1]

print("Policy Iteration:")
for gamma in gammas:
  _, deterministic_policy, deterministic_iterations = policy_iteration(P_deterministic, gamma)
  _, stochastic_policy, stochastic_iterations = policy_iteration(P_stochastic, gamma)
  print(f"""
  Gamma: {gamma}
  Środowiska deterministyczne
  Empiryczny wynik: {evaluate_empirically(env_deterministic, deterministic_policy)}
  Liczba iteracji: {deterministic_iterations}
  Polityka: {policy}
  Środowiska stochastycznego
  Empiryczny wynik: {evaluate_empirically(env_stochastic, stochastic_policy)}
  Liczba iteracji: {stochastic_iterations}
  Polityka: {policy}
  """)
print("Value Iteration:")
for gamma in gammas:
  _, deterministic_policy, deterministic_iterations = value_iteration(P_deterministic, gamma)
  _, stochastic_policy, stochastic_iterations = value_iteration(P_stochastic, gamma)
  print(f"""
  Gamma: {gamma}
  Środowiska deterministyczne
  Empiryczny wynik: {evaluate_empirically(env_deterministic, deterministic_policy)}
  Liczba iteracji: {deterministic_iterations}
  Polityka: {policy}
  Środowiska stochastycznego
  Empiryczny wynik: {evaluate_empirically(env_stochastic, stochastic_policy)}
  Liczba iteracji: {stochastic_iterations}
  Polityka: {policy}
  """)

Policy Iteration:

  Gamma: 0.0
  Środowiska deterministyczne
  Empiryczny wynik: 0.0
  Liczba iteracji: 2
  Polityka: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]
  Środowiska stochastycznego
  Empiryczny wynik: 0.0
  Liczba iteracji: 2
  Polityka: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]
  

  Gamma: 0.1
  Środowiska deterministyczne
  Empiryczny wynik: 1.0
  Liczba iteracji: 7
  Polityka: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]
  Środowiska stochastycznego
  Empiryczny wynik: 0.47400000000000037
  Liczba iteracji: 6
  Polityka: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]
  

  Gamma: 0.5
  Środowiska deterministyczne
  Empiryczny wynik: 1.0
  Liczba iteracji: 7
  Polityka: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]
  Środowiska stochastycznego
  Empiryczny wynik: 0.4430000000000003
  Liczba iteracji: 5
  Polityka: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]
  

  Gamma: 0.9
  Środowiska deterministyczne
  Empiryczny wynik: 1.0
  Liczba iteracji: 

## Zad. 2 - Monte Carlo (10 pkt.)
W komórce poniżej zaimplementuj metodę **On-policy Monte Carlo** dla polityki epsilon-greedy.
Zakładamy, że model przejść nie jest w tym wypadku dla nas dostępny,
dlatego możesz używać wyłącznie metod `env.reset()` i `env.step()`
w swojej implementacji, w celu wygenerowania nowego epizodu.

- Zaproponuj warunek stopu dla swojej implementacji.
- Jaki jest wpływ epsilony na działanie algorytmu?
- Jaka prosta modyfikacja nagród środowiska przyśpieszyłaby odkrywanie dobrej polityki? Zmodyfikuj env.P i zademonstruj.

Tip: z racji, że env.P jest dostępne, możesz porównać wyniki `on_policy_eps_greedy_monte_carlo` ze wynikami `value_iteration`. 

#### Uwaga: nie zmieniaj nazwy funkcji `on_policy_eps_greedy_monte_carlo`, ani jej pierwszych argumentów (możesz dodać nowe argumenty z wartościami domyślnymi). Nie dopisuj do komórki z funkcją innego kodu. Może zdefiniować funkcje pomocnicze dla funkcji w tej samej komórce (sprawdzarka wyciągnie ze zgłoszonego notebooka wyłącznie komórkę zawierającą funkcję `on_policy_eps_greedy_monte_carlo` do sprawdzenia, kod w innych komórkach nie będzie widziany przez sprawdzarkę!).

Odpowiedź:
1. Propozycją implementacji warunku stopu jest maksymalna liczba epizodów.
2. Epsilon wpływa na to, jak często algorytm eksploruje środowisko. Im większa epsilon, tym częściej eksploruje.
  - Przy mniejszych wartościach agent będzie częściej podejmował akcji według założonej polityki i rzadziej eksplorował środowisko. Co może skutkować tym, że algorytm będzie potrzebował mniej epizodów do pełnej polityki, jednak istnieje ryzyko, że algorytm utknie w jakiejś lokalnej optymalności.
  - Przy większych wartościach agent będzie częściej eksplorował środowisko i rzadziej podejmował akcje według założonej polityki. Co może skutkować tym, że algorytm będzie potrzebował więcej epizodów do odkrycia optymalnej polityki.
3. Modyfikacja nagród polega na tym, że istnieje kara za każdą wpadniętą dziurę. Dzięki temu algorytm szybciej znajduje optymalną politykę.

In [119]:
from gym.envs.toy_text import FrozenLakeEnv

def modify_rewards(env):
  P = env.P
  map_ = b''.join(item for sublist in env.desc for item in sublist)

  holes = [state for (state, type_) in enumerate(map_) if type_ is ord(b'H')]
  goals = [state for (state, type_) in enumerate(map_) if type_ is ord(b'G')]

  for state in P:
    for action in P[state]:
      for (i, (probability, next_state, reward, termination)) in enumerate(P[state][action]):
        if next_state in holes:
          P[state][action][i] = (probability, next_state, -0.05, termination)
        elif state != next_state and next_state in goals:
          P[state][action][i] = (probability, next_state, 1, termination)
        elif state != next_state:
          P[state][action][i] = (probability, next_state, 0, termination)

In [108]:
from collections import defaultdict
import random

def on_policy_eps_greedy_monte_carlo(env, eps, gamma, max_episodes=10000):
  """
  Argumenty:
      env — środowisko implementujące metody `reset()` oraz `step(action)`
      eps — współczynnik eksploracji
      gamma — współczynnik dyskontujący
  Zwracane wartości:
      Q — lista o długości len(P) zawierający listy z oszacowanymi wartościami dla stanu s i akcji a: Q[s][a]
      pi — lista o długości len(P) zawierający wyznaczoną deterministyczną (zachłanną) politykę - akcję dla stanu s: pi[s]
      i — ilość epizodów wygenerowanych przez algorytm
  """
  def generate_episodes():
    def e_greedy(state):
      if random.uniform(0, 1) < eps: return env.action_space.sample()
      return policy[state]

    nonlocal iterations
    while (iterations := iterations + 1) < max_episodes:
      state = env.reset()
      episode = []

      while True:
        action = e_greedy(state)
        (next_state, reward, termination, _) = env.step(action)
        episode.append((state, action, reward))
        state = next_state
        if termination: break
      yield episode

  visits = defaultdict(int)
  def evaluate(episode):
    returns = 0

    states = [state for (state, _, _) in episode]
    for (epoch, (state, action, reward)) in reversed(list(enumerate(episode))):
      returns = gamma * returns + reward

      if state not in states[:epoch]:
        visits[(state, action)] += 1
        Q[state][action] += (returns - Q[state][action]) / visits[(state, action)]

  def extract():
    return [Q[state].index(max(Q[state])) for state in range(len(Q))]

  P = env.P
  states = range(len(P))
  Q = [[0] * len(P[s]) for s in states]
  policy = extract()
  iterations = 0

  episode_it = generate_episodes()
  while episode := next(episode_it, None):
    evaluate(episode)
    policy = extract()

  return Q, policy, iterations


In [120]:
env: FrozenLakeEnv = gym.make('FrozenLake-v0', is_slippery=True)
epsilon = 0.25
gamma = 0.99
max_episodes = 100000
newline = '\n'
modify_rewards(env)
Q, policy, iterations = on_policy_eps_greedy_monte_carlo(env, epsilon, gamma, max_episodes)
print("Wartości stanów i akcji (Q):")
import numpy as np
env.render()
Q = np.round_(Q, decimals=2)
print(*Q, sep='\n')
print(f"""
Polityka (pi):
{policy}
Liczba epizodów: {iterations}
Wynik empiryczny: {evaluate_empirically(env, policy):.2f}
""")

Wartości stanów i akcji (Q):
  (Right)
SFFF
FHFH
FFFH
HFFG
[0.12 0.1  0.11 0.1 ]
[0.04 0.05 0.03 0.09]
[0.06 0.08 0.08 0.05]
[0.02 0.03 0.01 0.06]
[0.14 0.08 0.07 0.06]
[0. 0. 0. 0.]
[ 0.11  0.07  0.12 -0.01]
[0. 0. 0. 0.]
[0.09 0.13 0.12 0.19]
[0.18 0.3  0.22 0.15]
[0.35 0.3  0.23 0.12]
[0. 0. 0. 0.]
[0. 0. 0. 0.]
[0.21 0.33 0.45 0.29]
[0.46 0.69 0.64 0.57]
[0. 0. 0. 0.]

Polityka (pi):
[0, 3, 2, 3, 0, 0, 2, 0, 3, 1, 0, 0, 0, 2, 1, 0]
Liczba epizodów: 100000
Wynik empiryczny: 0.69

